In [62]:
# !pip install --upgrade google-api-python-client

In [63]:
# !pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [64]:
import sys
sys.path.append('../') 

from config.youtube import YOUTUBE_API_KEY

from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON
import urllib.request
import re

Los proyectos que habilitan la API de datos de YouTube tienen una asignación de cuota predeterminada de 10,000 unidades por día, una cantidad suficiente para la gran mayoría de los usuarios de la API. La cuota predeterminada, que está sujeta a cambios, nos ayuda a optimizar las asignaciones de cuotas y a escalar nuestra infraestructura de una manera que sea más significativa para nuestros usuarios de API. Puedes ver el uso de tu cuota en la página Cuotas en la Consola de API.

Nota: Si alcanzas el límite de cuota, puedes solicitar un aumento del cuota a través del formulario de solicitud de extensión de cuota para los servicios de la API de YouTube.

Calcula el uso de la cuota

Google calcula el uso de tu cuota mediante la asignación de un costo a cada solicitud. Los diferentes tipos de operaciones tienen diferentes costos de cuotas. Por ejemplo:

Una operación de lectura que recupera una lista de recursos (canales, videos o listas de reproducción) suele costar 1 unidad.
Una operación de escritura que crea, actualiza o borra un recurso suele tener un costo de 50 unidades.
Una solicitud de búsqueda cuesta 100 unidades.
La carga de un video cuesta 1600 unidades.

## Definición API

In [65]:
api_key = YOUTUBE_API_KEY
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key)

## Obtencion comentarios en videos del Canal

Buscar en el codigo fuente de la pagina para obtener el ID <br>
https://www.youtube.com/channel/

### Obtención de Id del canal

In [66]:
def extract_channel_ids(urls):
    all_channels = []
    for url in urls:
        with urllib.request.urlopen(url) as response:
            html_content = response.read().decode('utf-8')

        pattern = r'https://www.youtube.com/channel/([^"]+)'
        matches = re.findall(pattern, html_content)

        all_channels.append(matches[0])

    return all_channels

In [67]:
urls = [
    'https://www.youtube.com/@LaMMordida',
    #More channels
]
channel_ids = extract_channel_ids(urls)

In [68]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
                }
        all_data.append(data)

    return (pd.DataFrame(all_data))

In [69]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,La Manzana Mordida,1340000,253794221,3081,UUeNM2cr0tcisyZ15OtsNldg


### Obtencion videos del canal

In [70]:
def get_video_ids(youtube, playlist_id, limit=None, output_file="../data/yt_video_ids.txt"):
    video_ids = []
    next_page_token = True
    total_results = 0

    while next_page_token is not None and (limit is None or total_results < limit):
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50  # Máximo permitido por la API
        )
        response = request.execute()

        for item in response['items']:
            video_id = item['contentDetails']['videoId']
            video_ids.append(video_id)
            total_results += 1

            # Escribir el ID del video en el archivo de salida
            with open(output_file, "a") as file:
                file.write(video_id + "\n")

            if limit is not None and total_results == limit:
                break

        next_page_token = response.get('nextPageToken')

    return video_ids


In [71]:
video_ids = []
for index, row in channel_stats.iterrows():
    playlist_id = row['playlistId']
    video_ids.extend(get_video_ids(youtube, playlist_id, limit=1794))

video_ids

['lzwUPjxkmTY',
 'wW6Mt-PdCHM',
 'u_HBDVEYNVY',
 'EUzgwnpQKew',
 'nFGk0RjNkvw',
 'd0Tj7hDYZrw',
 'bOe3IxIwFxM',
 'zHWnaQg7B78',
 'yhZ-_Rb6ijc',
 'Y3fF8e2gXCo',
 'oYQ3bfEJVqc',
 'nuVo23kk5SA',
 'fAoiQD33Jfw',
 '_XH25aHMTjU',
 'IEmZiTx2s2s',
 '4jHCtXH4wv8',
 'wdgC4sgPpqY',
 'X9v-tj_FF0A',
 'ns_bYublOww',
 '5wuKKhZPSgE',
 'nzbHQqwdd8A',
 'YqcsrPYSDrM',
 'joIg0W6jYeU',
 '6XHLiLQD0kk',
 'qRHPNTzQRTk',
 'Z6qDlg3yFY0',
 '9CGTcsmGrbk',
 '_JajhT9Q_Zw',
 'k1miUYVs9TM',
 'yirD6umdRMs',
 'sdBa1bWb1Eo',
 'o7ltBlMeImU',
 'U479f4o52rY',
 'ZGGrnExcfvs',
 '2X4bEDHtmP4',
 'DBOh5KMrVyc',
 '-2LprocOud8',
 'LF5Ks46_8W8',
 'ks0CfD_0E-k',
 'R7ihIyesc2E',
 'IRPkCIACuY0',
 'ZRPuggejV4E',
 'HtJU2Gc14zs',
 '-Wbboa-uF4k',
 '9qczBgUQOcY',
 'w9QOeE4SJ-0',
 'Pe0OP0eY_Eg',
 'ucfd-N6KSWk',
 'obCoric5qCU',
 'TM0RnEqYEw8',
 'lzwUPjxkmTY',
 'wW6Mt-PdCHM',
 'u_HBDVEYNVY',
 'EUzgwnpQKew',
 'nFGk0RjNkvw',
 'd0Tj7hDYZrw',
 'bOe3IxIwFxM',
 'zHWnaQg7B78',
 'yhZ-_Rb6ijc',
 'Y3fF8e2gXCo',
 'oYQ3bfEJVqc',
 'nuVo23kk5SA',
 'fAoiQD

### Obtencion estadísticas de vídeos

In [72]:
def get_video_details(youtube, video_ids):
        all_video_info = []

        for i in range(0, len(video_ids), 50):
                request = youtube.videos().list(
                        part="snippet,contentDetails,statistics",
                        id=','.join(video_ids[i:i+50])
                )
                response = request.execute()

                for video in response['items']:
                        stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                                        'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                                        'contentDetails': ['duration', 'definition', 'caption']
                                        }
                        video_info = {}
                        video_info['video_id'] = video['id']

                        for k in stats_to_keep.keys():
                                for v in stats_to_keep[k]:
                                        try:
                                                video_info[v] = video[k][v]
                                        except:
                                                video_info[v] = None
                        
                        all_video_info.append(video_info)

        return pd.DataFrame(all_video_info)

In [73]:
#with open('../data/youtube/yt_video_ids.txt', 'r') as file:
    #video_ids = file.read().splitlines()

video_df = get_video_details(youtube, video_ids)
video_df.to_csv('../data/yt_video_stats.csv', sep=';', index=False)
video_df.head(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,lzwUPjxkmTY,La Manzana Mordida,BOMBAZO: TODO sobre los NUEVOS iPad Pro OLED y...,¡Prepárate para alucinar! En este video te tra...,"[iPad Pro OLED, iPad Air 12.9, filtraciones iP...",2024-04-15T16:00:38Z,2799,368,None,18,PT9M37S,hd,true
1,wW6Mt-PdCHM,La Manzana Mordida,La GRAN MEJORA que tendrá el Apple Watch Serie...,¿Quieres saber cuál será el gran avance del Ap...,"[Apple Watch Series 10, pantallas OLED Apple W...",2024-04-15T14:00:41Z,672,77,None,3,PT1M,hd,true
2,u_HBDVEYNVY,La Manzana Mordida,El MEJOR Android que COMPRARÍA si no tuviera i...,"Aunque soy usuario de iPhone, reconozco que ha...","[mejor Android, Android vs iPhone, Android fav...",2024-04-14T16:00:41Z,5975,291,None,5,PT59S,hd,true
3,EUzgwnpQKew,La Manzana Mordida,¿Puede Apple DERROTAR a Copilot Pro y Gemini c...,"En el mundo de la inteligencia artificial, gig...","[Apple IA, Apple inteligencia artificial, Appl...",2024-04-14T13:30:06Z,2815,142,None,3,PT1M,hd,true
4,nFGk0RjNkvw,La Manzana Mordida,"Fecha lanzamiento nuevos iPads, filtración iPh...",¡No te pierdas nuestro resumen semanal de noti...,"[Apple, noticias apple, ipad pro 2024, ipad ai...",2024-04-12T16:00:41Z,50721,2045,None,79,PT20M45S,hd,true


### Comentarios

In [74]:
def get_comments_in_videos(youtube, video_ids, limit=None):
    all_comments = []
    
    for video_id in video_ids:
        next_page_token = None  # Inicializamos el token de página para la paginación
        
        while True:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=100,  # Establecemos un límite máximo por página
                    pageToken=next_page_token
                )
                response = request.execute()
            
                comments_in_video = [
                    {
                        'comment': comment['snippet']['topLevelComment']['snippet']['textOriginal'],
                        'date': comment['snippet']['topLevelComment']['snippet']['publishedAt']
                    }
                    for comment in response.get('items', [])
                ]
                
                all_comments.extend(comments_in_video)
                
                if 'nextPageToken' in response:
                    next_page_token = response['nextPageToken']
                else:
                    break  # No hay más páginas, terminar el bucle de paginación
        
            except Exception as e:
                print(f"Could not get comments for video {video_id}: {str(e)}")
    
    return pd.DataFrame(all_comments)

In [75]:
#with open('../data/yt_video_ids.txt', 'r') as file:
    #video_ids = file.read().splitlines()

comments_df = get_comments_in_videos(youtube, video_ids , limit=None)
comments_df.to_csv('../data/yt_video_comments.csv', sep=';', index=False)
comments_df

,comment,date
0,Miedo me da los posibles precios de los nuevos...,2024-04-15T17:14:53Z
1,Si ponen el MagSafe detrás pueden saca una bas...,2024-04-15T17:13:26Z
2,"No me parece para tanto, muy continuista, apen...",2024-04-15T16:58:57Z
3,"Me quedo con mi Ipad pro 12,9 con M2",2024-04-15T16:54:00Z
4,"A medida que pasan los años, el hardware siemp...",2024-04-15T16:32:55Z
...,...,...
58947,"Que crees si me compro un iPad Mini 6th gen, t...",2024-03-19T13:26:45Z
58948,Podrían incorporar la app de freestyle para ir...,2024-03-17T10:15:01Z
58949,Yo creo que Apple nos a engañado con eso del ...,2024-03-16T23:03:00Z
58950,Si copian uno de la competencia y no pagan las...,2024-03-16T20:19:06Z
